In [4]:
f = open("../api_key.txt", "r")
api_key = f.readline()

In [2]:
import requests
import json
from os.path import exists
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import data_manager as dm

In [27]:
def predict_and_plot(symbol, api_key, model):
    x, y, time_series = dm.preprocess_data_cnn_1d(symbol, api_key)
    pred = model.predict(np.asarray(x))
    print(pred.shape())
    stock_data = dm.get_stock_data(symbol, api_key)
    plt.plot(pd.to_datetime(stock_data['Dates']), np.array(stock_data['Prices']).astype(float))
    plt.plot(pd.to_datetime(time_series), pred)
    plt.legend(["Actual Price", "Predicted Price"])
    plt.show()

In [12]:
def get_compiled_model():
    model = keras.Sequential()
    model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(365,1,)))
    model.add(layers.MaxPooling1D())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(loss='mean_squared_error',
                  metrics=[tf.keras.metrics.MeanSquaredError()],
                  optimizer='adam')
    return model

In [16]:
def train_model(symbol, model, api_key, start=None, end=None):
    if (start is None and end is not None) or (start is not None and end is None):
        raise ValueError("start and end has to be both non-None or both None")
    x = y = []
    x, y, time_series = dm.preprocess_data_cnn_1d(symbol, api_key)
    model.fit(np.asarray(x), np.asarray(y), epochs=30, batch_size=128)
    return model

In [25]:
model = get_compiled_model()
train_model('IBM', model, api_key)

Epoch 1/30
43/43 [==============================] - 2s 43ms/step - loss: 6915.6733 - mean_squared_error: 6915.6763
Epoch 2/30
43/43 [==============================] - 2s 39ms/step - loss: 2632.9727 - mean_squared_error: 2632.9722
Epoch 3/30
43/43 [==============================] - 2s 38ms/step - loss: 2610.6658 - mean_squared_error: 2610.6655
Epoch 4/30
 8/43 [====>.........................] - ETA: 1s - loss: 2487.7827 - mean_squared_error: 2487.7827

KeyboardInterrupt: 

In [28]:
predict_and_plot("META", api_key, model)

70/70 [==============================] - 0s 4ms/step


TypeError: 'tuple' object is not callable